# Edge Detection Algorithms

This jupyter notebook aims to present several edge detection algorithms, show their implementation in python and make a comparison between them.

Currently implemented algorithms include Sobel, Prewitt, Roberts, and Canny edge detectors.

In [1]:
from scipy.signal import convolve2d
from scipy import misc
import matplotlib.pyplot as plt
import numpy as np
import sys
%matplotlib inline

### Gaussian filter
Gaussian filter is responsible for smoothing out an image removing abrupt changes in neighbour pixels.

As all gradient based algorithms are very sensitive on noise and can have a huge performance change due to fluctuations and anomalies in images, it's a reasonable decision to apply gaussian filter on images before giving them to any edge detector. We could use any smoothing filter but Gaussian is shown to work good in practice.

Usually, for edge detection, people choose 5x5 gaussian filter with standard deviation=1.4, so that's why the default parameters of our function are `kernel_size=(5,5)`, `standard_deviation=1.4`

In [2]:
def gaussian_filter(kernel_size=(5,5), standard_deviation=1.4, normalize=True):
    """
    :returns 2D Gaussian mask of shape kernel_size
    """
    res = np.zeros(shape=kernel_size)
    centerI = ( kernel_size[0] - 1 ) / 2
    centerJ = ( kernel_size[1] - 1 ) / 2
    for i in range(kernel_size[0]):
        for j in range(kernel_size[1]):
            res[i][j] = np.exp( -( (i - centerI)**2. + (j - centerJ)**2. ) / ( 2. * standard_deviation**2. )) \
                        / ( 2. * np.pi * standard_deviation**2. )
    
    if normalize:
        res /= np.sum(res)
    return res

def smooth_gaussian(input_image):
    """Helper function for applying gaussian filter on an input image"""
    return convolve2d(input_image, gaussian_filter(), mode='same')

In [3]:
def apply_thresholding(input_image, threshold=70, high_value=255, low_value=0):
    result = np.copy(input_image)
    high_indices = result[:,:] > threshold
    low_indices = result[:,:] <= threshold
    result[high_indices] = high_value
    result[low_indices] = low_value
    return result

In [4]:
class EdgeDetector(object):
    """
    Base class for all edge detecting classes
    Any edge detector should implement an abstract function run_algorithm(input_image)
    """
    def __init__(self, smooth_image=None, apply_thresholding=None):
        self.smooth_image = smooth_image
        self.apply_thresholding = apply_thresholding
        
    def detect_edges(self, input_image):
        """
        Applies smoothing if it's not None
        runs algorithm defined by a subclass
        Applies thesholding if it's not None
        :returns result
        """
        if self.smooth_image is not None:
            input_image = self.smooth_image(input_image)
        
        result = self.run_algorithm(input_image)
        result -= result.min()
        if result.max() != 0:
            result *= 255./result.max()
        
        if self.apply_thresholding is not None:
            result = self.apply_thresholding(result)
        return result
    
    def run_algorithm(self, input_image):
        raise NotImplementedError('Please implement this method')

## First degree derivative based algorithms
### Operator based Edge detectors
As Sobel's, Prewitt's or  Robert's algortihms are based on applying a convolutional filter on the input image, they all share the same logic of processing the input. The only difference is the kernel matrix that is applied to an input image.

So, lets create a base class for this kind of edge detectors `OperatorEdgeDetector`

In [5]:
class OperatorEdgeDetector(EdgeDetector):
    def __init__(self, filterX, filterY, smooth_image=smooth_gaussian, thresholding=apply_thresholding):
        """
        :param filterX filter-matrix (kernel) to compute gradient on X direction
        :param filterY filter-matrix (kernel) to compute gradient on Y direction
        :param smooth_image a function that will smooth the image before doing any detection
                            the default is gaussian, but it can be any function (set to None if no filtering is needed)
        """
        super().__init__(smooth_image=smooth_image, apply_thresholding=thresholding)
        self.filterX = filterX
        self.filterY = filterY
        
    def get_gradients(self, input_image):
        """
        :returns gradients in X and Y directions
        """
        gradX = convolve2d(input_image, self.filterX, mode='same') # Convolve input image with filter-matrix in X direction
        gradY = convolve2d(input_image, self.filterY, mode='same') # Convolve input image with filter-matrix in Y direction
        return gradX, gradY
    
    def run_algorithm(self, input_image):
        """
        Computes gradient in X and Y directions (gradX and gradY respectively)
        And gets the result by combining this gradients with sqrt( gradX^2 + gradY^2 )
        :returns 2D matrix of edges 
            closer to 255 => edges are more visible
            closer to 0 => no edges there
        """
        
        gradX, gradY = self.get_gradients(input_image)
        result = np.sqrt( np.power(gradX, 2) + np.power(gradY, 2) )
        return result

### Sobel
Sobel's edge detector is a simple convolutional filter with kernel matrix

In X and Y directions respectively
```
-1 0 1             -1 -2 -1
-2 0 2              0 0 0
-1 0 1              1 2 1```

In [6]:
class SobelEdgeDetector(OperatorEdgeDetector):
    def __init__(self, smooth_image=smooth_gaussian, thresholding=apply_thresholding):
        filterX = np.array([[-1, 0, 1],
                            [-2, 0, 2],
                            [-1, 0, 1]], dtype=np.float32)
        filterY = np.array([[-1, -2, -1],
                            [0, 0, 0],
                            [1, 2, 1]], dtype=np.float32)
        super().__init__(filterX=filterX, filterY=filterY, smooth_image=smooth_image, thresholding=apply_thresholding)

### Prewitt
Prewitt's edge detector is also a simple convolutional filter with kernel matrix

In X and Y directions respectively
```
-1 0 1             -1 -1 -1
-1 0 1              0 0 0
-1 0 1              1 1 1```

In [7]:
class PrewittEdgeDetector(OperatorEdgeDetector):
    def __init__(self, smooth_image=smooth_gaussian, thresholding=apply_thresholding):
        filterX = np.array([[-1, 0, 1],
                            [-1, 0, 1],
                            [-1, 0, 1]], dtype=np.float32)
        filterY = np.array([[-1, -1, -1],
                            [0, 0, 0],
                            [1, 1, 1]], dtype=np.float32)
        super().__init__(filterX=filterX, filterY=filterY, smooth_image=smooth_image, thresholding=apply_thresholding)

### Robert
Robert's edge detector is also a simple convolutional filter with kernel matrix

In X and Y directions respectively
```
-1 0             0 -1
0 1              1 0```

In [8]:
class RobertsEdgeDetector(OperatorEdgeDetector):
    def __init__(self, smooth_image=smooth_gaussian, thresholding=apply_thresholding):
        filterX = np.array([[-1, 0],
                            [0, 1]], dtype=np.float32)
        filterY = np.array([[0, -1],
                            [1, 0]], dtype=np.float32)
        super().__init__(filterX=filterX, filterY=filterY, smooth_image=smooth_image, thresholding=apply_thresholding)

### Canny
Canny edge detection algorithm is an extention of any algorithm listed above. It refines the edges found with Sobel/Prewitt/or any other gradient based edge detector.

Canny edge detector applies thresholding to choose those edges that have enough intensity to consider them as a real edge and not a noise, after which comparing every pixel with its neighbours along the gradient direction to make sure that it's maximum among them. If not, the pixel is set to non-edge (black).

In [9]:
def double_thresholding(input_image, threshold_hight=200, threshold_low=70, high_value=255, low_value=0):
    result = np.copy(input_image)
    high_indices = result[:,:] > threshold_hight
    low_indices = result[:,:] < threshold_low
    result[high_indices] = high_value
    result[low_indices] = low_value
    return result

In [10]:
class CannyEdgeDetector(EdgeDetector):
    def __init__(self, 
                 smooth_image=smooth_gaussian, 
                 gradient_calculator=SobelEdgeDetector(), 
                 thresholding=double_thresholding):
        super().__init__(smooth_image, apply_thresholding=thresholding)
        self.gradient_calculator = gradient_calculator

    def filter_necessary(self, i, j, result, used, gradient, angle):
        def is_largest(cur_i, cur_j, neighbours):
            def is_larger(cur_i, cur_j, item_i, item_j):
                if item_i < 0 or item_i >= len(gradient):         return True
                if item_j < 0 or item_j >= len(gradient[item_i]): return True
                return gradient[cur_i][cur_j] >= gradient[item_i][item_j]
            
            return all([ is_larger(cur_i, cur_j, item_i, item_j) for (item_i, item_j) in neighbours])

        
        def update_result(cur_i, cur_j, neighbours):
            if is_largest(cur_i, cur_j, neighbours):   result[i][j] = 255
            else:                                      result[i][j] = 0
        
        if i < 0 or i >= len(result):    return
        if j < 0 or j >= len(result[i]): return
        if used[i][j]:                   return
        used[i][j] = True
        
        if self.threshold_low <= result[i][j] <= self.threshold_high:
            if angle[i][j] == 0:          update_result(i, j, [(i, j-1), (i, j+1)])
            if angle[i][j] == np.pi/4:    update_result(i, j, [(i-1, j+1), (i+1, j-1)])
            if angle[i][j] == np.pi/2:    update_result(i, j, [(i-1, j), (i+1, j)])
            if angle[i][j] == np.pi*3/4:  update_result(i, j, [(i, j+1), (i+1, j+1)])
    
        if result[i][j] == 255:
            self.filter_necessary(i+1, j, result, used, gradient, angle)
            self.filter_necessary(i-1, j, result, used, gradient, angle)
            self.filter_necessary(i, j+1, result, used, gradient, angle)
            self.filter_necessary(i, j-1, result, used, gradient, angle)
        
    def run_algorithm(self, input_image):
        if self.smooth_image is not None:
            input_image = self.smooth_image(input_image)
        
        gradX, gradY = self.gradient_calculator.get_gradients(input_image)
        gradient = np.sqrt( np.power(gradX, 2), np.power(gradY, 2) )
        result = np.copy(gradient)
        angle = np.arctan2(gradY, gradX)
        
        def normalize_angle(val):
            val = val if val > 0 else val + 2*np.pi
            val = min([np.pi*0/4, np.pi*1/4, np.pi*2/4, np.pi*3/4,
                                   np.pi*4/4, np.pi*5/4, np.pi*6/4, np.pi*7/4, 
                                   np.pi*8/4], key=lambda x: abs(x-val))
            if val == np.pi*2 or val == np.pi: val = 0
            if val > np.pi: val -= np.pi
            return val
        
        for i in range(len(angle)):
            for j in range(len(angle[i])):
                angle[i][j] = normalize_angle( angle[i][j] )
        
        idx = result[:,:] > self.threshold_high
        result[idx] = 255
        idx = result[:,:] < self.threshold_low
        result[idx] = 0
        
        used = np.zeros(shape=gradient.shape)
        for i in range(len(used)):
            for j in range(len(used[i])):
                if result[i][j] == 255:
                    self.filter_necessary(i, j, result, used, gradient, angle)
                    
        idx = result[:,:] != 255
        result[idx] = 0
        return result

## Second degree derivative based algorithms
### Laplacian
To find edges with second order derivative we need to find all intersections with 0 of the function. In this case our function is considered to be each `3x3` sum-matrix of the input image colvolved by the `laplacian filter`.

If the center of `3x3` sub-matrix is greater than 0 and there is an element in that matrix that is smaller than 0 => there was a 0 intersection. Also if the center is smaller than 0 and there is an element that is larger than 0 => there also was an intersection with 0. To have a valid edge we also need to take into account that minimum and maximum elements of the `3x3` matrix are different enough (i.e. max - min > threshold)

If all the conditions above are satisfied then the center of `3x3` matrix is considered part of an edge

In [11]:
class LaplacianEdgeDetector(EdgeDetector):
    def __init__(self, laplacian_filter=None, smooth_image=smooth_gaussian):
        """
        :param laplacian_filter 2D np.array of values representing laplacian filter
        """
        super().__init__(smooth_image=smooth_image, apply_thresholding=None)
        if laplacian_filter is None:
            self.laplacian_filter = np.array([[0, 1, 0],
                                              [1, -4, 1],
                                              [0, 1, 0]], dtype=np.float32)
        else:
            self.laplacian_filter = laplacian_filter
        
    def run_algorithm(self, input_image, threshold=25):
        f = convolve2d(input_image, self.laplacian_filter, mode='same')
        result = np.zeros(shape=f.shape, dtype=np.float32)

        h = result.shape[0] # Height of the result
        w = result.shape[1] # Width of the result
        for i in range(1, h-1):
            for j in range(1, w-1):
                patch = f[i-1:i+2, j-1:j+2]            # Take the neighbour square of edge-size 3
                maxP = patch.max()                     # Compute maximum of that square
                minP = patch.min()                     # Compute minimum of that square
                if f[i][j] > 0:  zeroCross = minP < 0  # If there is an element lower than 0 and f[i][j] > 0
                else:            zeroCross = maxP > 0  # If there is an element greater than 0 and f[i][j] < 0
                    
                # If the difference between min and max elements is not too low and there is a zeroCross => it's an edge
                if maxP - minP > threshold and zeroCross:
                    result[i][j] = 255
        return result

## Comparison of the algorithms

In [12]:
def print_detections(input_image, detectors):
    
    def show(ax, image, label):
        ax.imshow(image, cmap='gray')
        ax.set_title(label)
        ax.axis('off')
        
    f, axis = plt.subplots(1, len(detectors)+1)
    f.set_figheight(15)
    f.set_figwidth(15)
    
    show(axis[0], input_image, 'Original')    
    for i, (edge_detector, label) in enumerate(detectors):
        out = edge_detector.detect_edges(input_image)
        show(axis[i+1], out, label)

In [ ]:
img = misc.imread('./test_image.jpg', mode='L')
sobel = SobelEdgeDetector()
roberts = RobertsEdgeDetector()
prewitt = PrewittEdgeDetector()
laplace = LaplacianEdgeDetector()

print_detections(img, [(sobel, 'Sobel'), (prewitt, 'Prewitt'), (roberts, 'Roberts'), (laplace, 'Laplacian')])

In [ ]:
canny = CannyEdgeDetector(threshold_high=150, threshold_low=70)
out = canny.detect_edges(img)
plt.imshow(out, cmap='gray')